In [5]:
import chromadb
import os
from dotenv import load_dotenv
import chromadb.utils.embedding_functions as embedding_functions
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pypdf, pypdf.filters
pypdf.filters.ZLIB_MAX_OUTPUT_LENGTH = 0

## Loads files and parses them individually into pages

In [6]:
folder_path = "/Users/garincurtis/Documents/Front End Development/climate_projections/chromaDB_scripts/documents"

documents = []

for file in os.listdir(folder_path): 
    if file.endswith('.pdf'):
        pdf_path = os.path.join(folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
        print(pdf_path)



print(len(documents))

/Users/garincurtis/Documents/Front End Development/climate_projections/chromaDB_scripts/documents/s41597-025-06200-4.pdf
/Users/garincurtis/Documents/Front End Development/climate_projections/chromaDB_scripts/documents/s41597-024-04188-x.pdf
/Users/garincurtis/Documents/Front End Development/climate_projections/chromaDB_scripts/documents/NEX-GDDP-CMIP6-Tech_Note.pdf
/Users/garincurtis/Documents/Front End Development/climate_projections/chromaDB_scripts/documents/Xu_2023_Environ._Res._Lett._18_044021.pdf
/Users/garincurtis/Documents/Front End Development/climate_projections/chromaDB_scripts/documents/hydrology-10-00150.pdf
/Users/garincurtis/Documents/Front End Development/climate_projections/chromaDB_scripts/documents/Earth s Future - 2022 - Park - What Does Global Land Climate Look Like at 2 C Warming.pdf
/Users/garincurtis/Documents/Front End Development/climate_projections/chromaDB_scripts/documents/s41598-025-13221-x.pdf
/Users/garincurtis/Documents/Front End Development/climate_pr

## Splits each page into chunks 

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

all_splits = text_splitter.split_documents(documents)

print(len(all_splits))



6201


In [22]:
print(all_splits[4006].page_content)
print(all_splits[4006].metadata)

12 
 
4.3 Future precipitation changes in China 
Fig. 5 shows projected changes in mean annual precipitation across China and its geological environmental zones under three 255 
climate scenarios and future periods, relative to the baseline. Overall, multimodel ensemble simulations show that future mean 
annual precipitation across China exhibits an increasing trend. The relative increases and associated uncertainties are great er 
under the SSP5 -8.5 than under the SSP1 -2.6 and SSP2 -4.5. Compared to the near -term (2026–2050) and mid -term future 
(2051–2075), the long -term future (2075 –2100) generally shows larger increases. According to projections from the NEX -
GDDP-CMIP6 multimodel ensemble, mean annual precipitation in China is expected to increase by 59 mm, 63 mm, and 111 260 
mm under the SSP1 -2.6, SSP2 -4.5, and SSP5 -8.5, respectively, during the long -term future from 2076 to 2100. These 
correspond to increases of 10.46%, 11.18%, and 19.14% relative to the baseline.
{

In [18]:
load_dotenv()

api_key = os.getenv("CHROMA_API_KEY")
tenant = os.getenv("CHROMA_TENANT")
database = os.getenv("DATABASE")

chroma_client = chromadb.CloudClient(
  api_key=api_key,
  tenant=tenant,
  database=database
)

In [19]:
load_dotenv()
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")
import chromadb.utils.embedding_functions as embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=CHROMA_OPENAI_API_KEY,
                model_name="text-embedding-3-large"
            )

In [20]:
collection = chroma_client.get_or_create_collection(
    name="climate_collection",
    embedding_function=openai_ef
)

In [23]:
for idx, chunk in enumerate (all_splits):
    collection.add(
        ids=[f"chunk_{idx}"],
        documents=[chunk.page_content],
        metadatas=[chunk.metadata] 
    )